# Pilot CFM implementation

### Load example programs


In [1]:
from example.python.crash import crashme_tup, crashme3_tup
from example.python.triangle import triangle_tup, triangle3_tup
from example.python.count import count_tup

import inspect

### Fuzzing configuration


In [2]:
program, seed_input_list = crashme3_tup
print(inspect.getsource(program))
try:
    program(seed_input_list[1])
except Exception as e:
    print(e)

n = 10000

def crashme3(s):
    if len(s) != 15:
        raise Exception("Wrong input.")
    if s[0] == "1":
        crashme(s[3:7])
    elif s[1] == "1":
        crashme(s[7:11])
    elif s[2] == "1":
        crashme(s[11:15])
    else:
        raise Exception("Wrong input.")

Deep bug!


### Run fuzzer and record input, coverage


In [3]:
from GreyboxFuzzer import GreyboxFuzzerRecorder
from MutationFuzzer import Mutator, PowerSchedule
from Coverage import FunctionContCovRunner

greybox_recorder = GreyboxFuzzerRecorder(
    seed_input_list, Mutator(), PowerSchedule()
)
runner = FunctionContCovRunner(program)
greybox_recorder.runs(runner, trials=n)
record = greybox_recorder.get_record()

print(f"Unique path: {len(record)}\n")
for path, inputs in list(record.items())[:3]:
    print(f"{path=}")
    print(f"{inputs=}")
    print()

Unique path: 17

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 14)), ('', ('crashme3', 16)), ('', ('crashme3', 19)), ('', ('__exit__', 83)))
inputs={'000abcdefgi#jkl', '23asdffrd!QSwer', '000abcdefghijkl', '03`dq~w`Er"ad!a', '000abcduf\x10ghijl', '>50;pdqI$s$fqwu', '000abJcdefghikl', '000abcdeFghijk,', '00bad\x01as$fqwe[r', '0\x110asdfad!ywerh'}

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 13)), ('crashme3:13', ('crashme', 2)), ('crashme3:13', ('crashme', 3)), ('crashme3:13', ('crashme', 4)), ('crashme3:13', ('crashme', 5)), ('crashme3:13', ('crashme', 6)), ('', ('__exit__', 83)))
inputs={'100bad!es6df7er', '100bad!asd2fqer', '100bad!asdfqwer', '100bad!asdqwenr', '100bad!,asdfq7g', '100bad!asdqlwer', '1g0bad!asdfqwer', '100bad!adfUqwer', '10"bad!a&sfqwer', '100bad!adfqweKr'}

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 14)), ('', ('crashme3', 15)), ('crashme3:15', ('crashme', 2)), ('crashme3:15', ('crashm

### Create a graph from coverage

In [4]:
from graph import Graph

contcov_graph = Graph()
coverage_graph = Graph()
for path, inputs in record.items():
    contcov_graph.accept(path)
    coverage_list = [contcov[1] for contcov in path]
    coverage_graph.accept(coverage_list)
# contcov_graph.print_graph("contcov.gv")
# coverage_graph.print_graph("coverage.gv")

In [5]:
from ControlFlowModel import ControlFlowModel

CFM = ControlFlowModel(record, contcov_graph, coverage_graph, runner)
CFM.identify_branch()
CFM.identify_call_context()

branches: [<('crashme3', 10)>, <('crashme3', 12)>, <('crashme3', 14)>, <('crashme3', 16)>, <('crashme', 2)>, <('crashme', 3)>, <('crashme', 4)>, <('crashme', 5)>]
call contexts: {'crashme3:17', 'crashme3:15', 'crashme3:13'}


### [Temp] assign temporary contextmap -- need to be changed to learn this

In [6]:
CFM.model_context(mut_trial=10)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bohrok/.pyenv/versions/3.9.7/envs/fuzzingbook/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/t1/6ty9rj2159ggbpcnm4zqcgs00000gn/T/9ab62733d3bd462b9d1c5ef6416a44ee-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/t1/6ty9rj2159ggbpcnm4zqcgs00000gn/T/9ab62733d3bd462b9d1c5ef6416a44ee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 42 RHS
At line 55 BOUNDS
At line 62 ENDATA
Problem MODEL has 12 rows, 6 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-12) rows, 0 (-6) columns and 0 (-18) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for 

/Users/bohrok/.pyenv/versions/3.9.7/envs/fuzzingbook/lib/python3.9/site-packages/pulp/pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Model formula

In [7]:
CFM.model_condition(max_trial=10)

Modeling <('crashme3', 10)> -> <('crashme3', 12)>... formula: <lambda x: '' <= x[14] <= '󥼥'> (conf: 0.9058823529411765, trial: (10 / 10))
Modeling <('crashme3', 10)> -> <('crashme3', 11)>... formula: <lambda: x x[1] < '' or x[1] > '񱮋'> (conf: 0.8823529411764706, trial: (10 / 10))
Modeling <('crashme3', 12)> -> <('crashme3', 14)>... formula: <lambda x: x[6] >= 'f'> (conf: 0.5375, trial: (10 / 10))
Modeling <('crashme3', 12)> -> <('crashme3', 13)>... formula: <lambda x: 'p' <= x[14] <= '󈼔'> (conf: 0.675, trial: (10 / 10))
Modeling <('crashme3', 14)> -> <('crashme3', 16)>... formula: <lambda x: x[9] > 'd'> (conf: 0.5818181818181818, trial: (10 / 10))
Modeling <('crashme3', 14)> -> <('crashme3', 15)>... formula: <lambda x: x[2] < '1'> (conf: 0.7090909090909091, trial: (10 / 10))
Modeling <('crashme3', 16)> -> <('crashme3', 19)>... formula: <lambda x: x[2] > x[11]> (conf: 0.7333333333333333, trial: (10 / 10))
Modeling <('crashme3', 16)> -> <('crashme3', 17)>... formula: <lambda x: '' <= 

In [8]:
CFM.get_context_map()

{'crashme3:13': range(3, 7),
 'crashme3:17': range(11, 15),
 'crashme3:15': range(7, 11)}

In [9]:
for edge, cond in CFM.get_edge_cond().items():
    print(f"edge={edge} {cond=}")

edge=E<('crashme3', 10) -> ('crashme3', 12)> cond=("lambda x: '\x15' <= x[14] <= '\U000e5f25'", 0.9058823529411765)
edge=E<('crashme3', 10) -> ('crashme3', 11)> cond=("lambda: x x[1] < '\x04' or x[1] > '\U00071b8b'", 0.8823529411764706)
edge=E<('crashme3', 12) -> ('crashme3', 14)> cond=("lambda x: x[6] >= 'f'", 0.5375)
edge=E<('crashme3', 12) -> ('crashme3', 13)> cond=("lambda x: 'p' <= x[14] <= '\U000c8f14'", 0.675)
edge=E<('crashme3', 14) -> ('crashme3', 16)> cond=("lambda x: x[9] > 'd'", 0.5818181818181818)
edge=E<('crashme3', 14) -> ('crashme3', 15)> cond=("lambda x: x[2] < '1'", 0.7090909090909091)
edge=E<('crashme3', 16) -> ('crashme3', 19)> cond=('lambda x: x[2] > x[11]', 0.7333333333333333)
edge=E<('crashme3', 16) -> ('crashme3', 17)> cond=("lambda x: '\x01' <= x[12] <= '\U000f78e6'", 0.6666666666666666)
edge=E<('crashme', 2) -> ('crashme', 3)> cond=("lambda x: x[1] != 'b'", 0.6491228070175439)
edge=E<('crashme', 2) -> ('__exit__', 83)> cond=(None, None)
edge=E<('crashme', 3) -